In [1]:
import numpy as np
import scipy #only sparse matrices allowed
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier

In [22]:
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
lightgbm_clf = LGBMClassifier()
lightgbm_clf.fit(X_train,y_train.ravel())
lightgbm_predictions = lightgbm_clf.predict(X_test)
lightgbm_accuracy = lightgbm_clf.score(X_test, y_test.ravel())

In [23]:
lightgbm_accuracy

0.8915680473372781

In [4]:
#https://stackoverflow.com/questions/3518778/how-do-i-read-csv-data-into-a-record-array-in-numpy
from numpy import genfromtxt
X_test_std = genfromtxt('X_test_std.csv', delimiter=',')
X_train_ada_std = genfromtxt('X_train_ada_std.csv', delimiter=',')
X_train_std = genfromtxt('X_train_std.csv', delimiter=',')

y_test = genfromtxt('y_test.csv', delimiter=',')
y_train_ada = genfromtxt('y_train_ada.csv', delimiter=',')
y_train = genfromtxt('y_train.csv', delimiter=',')

# Gridsearch for best hyperparameters:

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [6]:
#https://www.analyticsvidhya.com/blog/2021/06/tune-hyperparameters-with-gridsearchcv/
#https://www.mygreatlearning.com/blog/gridsearchcv/

In [7]:
param_grid = {
    'boosting_type' : ['gbdt', 'dart', 'goss'],
    'num_leaves' : [16, 31, 64],
    'max_depth' : [-1, 2, 4, 8],
    'n_estimators' : [50, 100, 200],
    'objective' : ['multiclass'],
    'class_weight' : ['balanced', None],
    'reg_alpha' : [0.0, 1.0, 0.1, 0.001],
    'reg_lambda' : [0.0, 1.0, 0.1, 0.001],
    'importance_type' : ['split', 'gain']
}

In [8]:
lightgbm_clf = LGBMClassifier()
grid = HalvingRandomSearchCV(estimator = lightgbm_clf, param_distributions = param_grid, refit = True, verbose = 2, n_jobs=-1, factor = 2)
# fitting the model for grid search 
grid.fit(X_train_ada_std, y_train_ada.ravel())

n_iterations: 11
n_required_iterations: 11
n_possible_iterations: 11
min_resources_: 30
max_resources_: 47961
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 1598
n_resources: 30
Fitting 5 folds for each of 1598 candidates, totalling 7990 fits
----------
iter: 1
n_candidates: 799
n_resources: 60
Fitting 5 folds for each of 799 candidates, totalling 3995 fits
----------
iter: 2
n_candidates: 400
n_resources: 120
Fitting 5 folds for each of 400 candidates, totalling 2000 fits
----------
iter: 3
n_candidates: 200
n_resources: 240
Fitting 5 folds for each of 200 candidates, totalling 1000 fits
----------
iter: 4
n_candidates: 100
n_resources: 480
Fitting 5 folds for each of 100 candidates, totalling 500 fits
----------
iter: 5
n_candidates: 50
n_resources: 960
Fitting 5 folds for each of 50 candidates, totalling 250 fits
----------
iter: 6
n_candidates: 25
n_resources: 1920
Fitting 5 folds for each of 25 candidates, totalling 125 fits
----------
iter: 7
n_candidate

HalvingRandomSearchCV(estimator=LGBMClassifier(), factor=2, n_jobs=-1,
                      param_distributions={'boosting_type': ['gbdt', 'dart',
                                                             'goss'],
                                           'class_weight': ['balanced', None],
                                           'importance_type': ['split', 'gain'],
                                           'max_depth': [-1, 2, 4, 8],
                                           'n_estimators': [50, 100, 200],
                                           'num_leaves': [16, 31, 64],
                                           'objective': ['multiclass'],
                                           'reg_alpha': [0.0, 1.0, 0.1, 0.001],
                                           'reg_lambda': [0.0, 1.0, 0.1,
                                                          0.001]},
                      verbose=2)

In [9]:
# print best parameter after tuning 
print(grid.best_params_) 
grid_predictions = grid.predict(X_test_std)

{'reg_lambda': 0.0, 'reg_alpha': 0.001, 'objective': 'multiclass', 'num_leaves': 64, 'n_estimators': 200, 'max_depth': -1, 'importance_type': 'split', 'class_weight': 'balanced', 'boosting_type': 'gbdt'}


In [10]:
print(classification_report(y_test, grid_predictions)) 

              precision    recall  f1-score   support

         0.0       0.99      0.76      0.86      2827
         1.0       0.23      0.97      0.37       109
         2.0       0.58      0.98      0.73       458

    accuracy                           0.80      3394
   macro avg       0.60      0.90      0.65      3394
weighted avg       0.91      0.80      0.83      3394



In [31]:
param_grid2 = {
    'boosting_type' : ['gbdt','dart', 'goss'],
    'num_leaves' : [16, 31, 64],
    'max_depth' : [-1, 2, 4, 8],
    'n_estimators' : [50, 100, 200],
    #'objective' : ['multiclass'],
    'class_weight' : ['balanced', None],
    'reg_alpha' : [0.0, 1.0, 0.1, 0.001],
    'reg_lambda' : [0.0, 1.0, 0.1, 0.001],
    'importance_type' : ['split', 'gain']
}

In [32]:
lightgbm_clf = LGBMClassifier()
grid = HalvingRandomSearchCV(estimator = lightgbm_clf, param_distributions = param_grid2, refit = True, verbose = 2, n_jobs=-1, factor = 2)
# fitting the model for grid search 
grid.fit(X_train_std, y_train.ravel())

n_iterations: 10
n_required_iterations: 10
n_possible_iterations: 10
min_resources_: 30
max_resources_: 19256
aggressive_elimination: False
factor: 2
----------
iter: 0
n_candidates: 641
n_resources: 30
Fitting 5 folds for each of 641 candidates, totalling 3205 fits
----------
iter: 1
n_candidates: 321
n_resources: 60
Fitting 5 folds for each of 321 candidates, totalling 1605 fits
----------
iter: 2
n_candidates: 161
n_resources: 120
Fitting 5 folds for each of 161 candidates, totalling 805 fits
----------
iter: 3
n_candidates: 81
n_resources: 240
Fitting 5 folds for each of 81 candidates, totalling 405 fits
----------
iter: 4
n_candidates: 41
n_resources: 480
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 5
n_candidates: 21
n_resources: 960
Fitting 5 folds for each of 21 candidates, totalling 105 fits
----------
iter: 6
n_candidates: 11
n_resources: 1920
Fitting 5 folds for each of 11 candidates, totalling 55 fits
----------
iter: 7
n_candidates: 6
n_re

HalvingRandomSearchCV(estimator=LGBMClassifier(), factor=2, n_jobs=-1,
                      param_distributions={'boosting_type': ['gbdt', 'dart',
                                                             'goss'],
                                           'class_weight': ['balanced', None],
                                           'importance_type': ['split', 'gain'],
                                           'max_depth': [-1, 2, 4, 8],
                                           'n_estimators': [50, 100, 200],
                                           'num_leaves': [16, 31, 64],
                                           'reg_alpha': [0.0, 1.0, 0.1, 0.001],
                                           'reg_lambda': [0.0, 1.0, 0.1,
                                                          0.001]},
                      verbose=2)

In [33]:
# print best parameter after tuning 
print(grid.best_params_) 
grid_predictions = grid.predict(X_test_std)

{'reg_lambda': 1.0, 'reg_alpha': 0.001, 'num_leaves': 64, 'n_estimators': 200, 'max_depth': 8, 'importance_type': 'gain', 'class_weight': None, 'boosting_type': 'gbdt'}


In [34]:
print(classification_report(y_test, grid_predictions)) 

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      2827
         1.0       1.00      0.94      0.97       109
         2.0       0.99      0.98      0.99       458

    accuracy                           0.99      3394
   macro avg       1.00      0.97      0.98      3394
weighted avg       0.99      0.99      0.99      3394



# Creating a csv for submission:

In [45]:
standardized_df_submit = df_standardizer(submit_df)

In [46]:
df_submit_np = standardized_df_submit.to_numpy()
num_submit = df_submit_np.shape[0]
X_submit = np.hstack((np.ones((num_submit,1)),df_submit_np))      

In [47]:
X_submit.shape

(7320, 22)

In [49]:
predictions_submit = lightgbm_clf.predict(X_submit)

In [50]:
#https://www.kite.com/python/answers/how-to-create-pandas-dataframe-from-a-numpy-array-in-python
predictions_df = pd.DataFrame(data=predictions_submit,columns=["LABELS"]).astype(int)
#https://stackoverflow.com/questions/18022845/pandas-index-column-title-or-name
predictions_df.index.name = "S.No"
predictions_df.to_csv(r"./ift3395-6390-weatherevents/submission.csv")

In [19]:
# TO DO